In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
df=pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')

In [3]:
# df.info()

In [4]:
def print_column_details(df):
    for col in df.columns:
        print(f"Column name: {col}")
        print(f"Column data type: {df[col].dtype}")
        print(f"Number of non-null values: {df[col].count()}")
        print(f"Number of unique values: {df[col].nunique()}")
        print()


In [5]:
def handle_missing_values(df):
    # Check for missing values in each column
    missing_values = df.isnull().sum()
    
    # Fill missing values according to data type
    for column, num_missing in missing_values.items():
        if num_missing > 0:
            if df[column].dtype == 'object':
                # Fill missing values in categorical columns with the most frequent value
                df[column].fillna(df[column].mode()[0], inplace=True)
            else:
                # Fill missing values in numerical columns with the median value
                df[column].fillna(df[column].median(), inplace=True)
                
    return df


In [6]:
handle_missing_values(df)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Grvl,Reg,Lvl,AllPub,...,0,Gd,MnPrv,Shed,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Grvl,Reg,Lvl,AllPub,...,0,Gd,MnPrv,Shed,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,Grvl,IR1,Lvl,AllPub,...,0,Gd,MnPrv,Shed,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,Grvl,IR1,Lvl,AllPub,...,0,Gd,MnPrv,Shed,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,Grvl,IR1,Lvl,AllPub,...,0,Gd,MnPrv,Shed,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,Grvl,Reg,Lvl,AllPub,...,0,Gd,MnPrv,Shed,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,Grvl,Reg,Lvl,AllPub,...,0,Gd,MnPrv,Shed,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,Grvl,Reg,Lvl,AllPub,...,0,Gd,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,Grvl,Reg,Lvl,AllPub,...,0,Gd,MnPrv,Shed,0,4,2010,WD,Normal,142125


In [7]:
df.isnull().sum()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 81, dtype: int64

In [8]:
def create_square_footage_feature(df):
    # Create a new column for total square footage
    df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']
    
    return df
create_square_footage_feature(df)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,TotalSF
0,1,60,RL,65.0,8450,Pave,Grvl,Reg,Lvl,AllPub,...,Gd,MnPrv,Shed,0,2,2008,WD,Normal,208500,2566
1,2,20,RL,80.0,9600,Pave,Grvl,Reg,Lvl,AllPub,...,Gd,MnPrv,Shed,0,5,2007,WD,Normal,181500,2524
2,3,60,RL,68.0,11250,Pave,Grvl,IR1,Lvl,AllPub,...,Gd,MnPrv,Shed,0,9,2008,WD,Normal,223500,2706
3,4,70,RL,60.0,9550,Pave,Grvl,IR1,Lvl,AllPub,...,Gd,MnPrv,Shed,0,2,2006,WD,Abnorml,140000,2473
4,5,60,RL,84.0,14260,Pave,Grvl,IR1,Lvl,AllPub,...,Gd,MnPrv,Shed,0,12,2008,WD,Normal,250000,3343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,Grvl,Reg,Lvl,AllPub,...,Gd,MnPrv,Shed,0,8,2007,WD,Normal,175000,2600
1456,1457,20,RL,85.0,13175,Pave,Grvl,Reg,Lvl,AllPub,...,Gd,MnPrv,Shed,0,2,2010,WD,Normal,210000,3615
1457,1458,70,RL,66.0,9042,Pave,Grvl,Reg,Lvl,AllPub,...,Gd,GdPrv,Shed,2500,5,2010,WD,Normal,266500,3492
1458,1459,20,RL,68.0,9717,Pave,Grvl,Reg,Lvl,AllPub,...,Gd,MnPrv,Shed,0,4,2010,WD,Normal,142125,2156


In [9]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from scipy import stats

def advanced_preprocess_numeric_columns(df, method='median'):
    """Takes a Pandas DataFrame and performs advanced preprocessing on all numeric columns that need it.
    
    Fills missing values using SimpleImputer, scales the data using StandardScaler, and removes outliers using the Z-score.
    
    Returns the modified DataFrame.
    """
    # Make a copy of the DataFrame to avoid modifying the original data
    df = df.copy()
    
    # Get a list of all numeric columns
    numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
    
    # Loop through each numeric column
    for col in numeric_columns:
        # Check if the column has missing values or outliers
        if df[col].isnull().any() or df[col].between(df[col].quantile(.01), df[col].quantile(.99)).all():
            # Fill missing values using SimpleImputer
            imputer = SimpleImputer(strategy=method)
            df[col] = imputer.fit_transform(df[col].values.reshape(-1, 1))

            # Scale the data using StandardScaler
            scaler = StandardScaler()
            df[col] = scaler.fit_transform(df[col].values.reshape(-1, 1))

            # Remove outliers using the Z-score
            z_scores = stats.zscore(df[col])
            df = df[np.abs(z_scores) < 3]
            
    print("Advanced preprocessing steps completed for columns that needed it: filled missing values, scaled data, and removed outliers.")
    return df

In [10]:


# Create a new dataframe with the modified data
modified_df = pd.DataFrame(advanced_preprocess_numeric_columns(df, method='median'))


Advanced preprocessing steps completed for columns that needed it: filled missing values, scaled data, and removed outliers.


In [11]:
modified_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,TotalSF
0,1,0.073375,RL,65.0,8450,Pave,Grvl,Reg,Lvl,AllPub,...,Gd,MnPrv,Shed,0,-1.608970,0.136651,WD,Normal,208500,2566
1,2,-0.872563,RL,80.0,9600,Pave,Grvl,Reg,Lvl,AllPub,...,Gd,MnPrv,Shed,0,-0.496922,-0.617833,WD,Normal,181500,2524
2,3,0.073375,RL,68.0,11250,Pave,Grvl,IR1,Lvl,AllPub,...,Gd,MnPrv,Shed,0,0.985808,0.136651,WD,Normal,223500,2706
3,4,0.309859,RL,60.0,9550,Pave,Grvl,IR1,Lvl,AllPub,...,Gd,MnPrv,Shed,0,-1.608970,-1.372318,WD,Abnorml,140000,2473
4,5,0.073375,RL,84.0,14260,Pave,Grvl,IR1,Lvl,AllPub,...,Gd,MnPrv,Shed,0,2.097856,0.136651,WD,Normal,250000,3343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,0.073375,RL,62.0,7917,Pave,Grvl,Reg,Lvl,AllPub,...,Gd,MnPrv,Shed,0,0.615126,-0.617833,WD,Normal,175000,2600
1456,1457,-0.872563,RL,85.0,13175,Pave,Grvl,Reg,Lvl,AllPub,...,Gd,MnPrv,Shed,0,-1.608970,1.645621,WD,Normal,210000,3615
1457,1458,0.309859,RL,66.0,9042,Pave,Grvl,Reg,Lvl,AllPub,...,Gd,GdPrv,Shed,2500,-0.496922,1.645621,WD,Normal,266500,3492
1458,1459,-0.872563,RL,68.0,9717,Pave,Grvl,Reg,Lvl,AllPub,...,Gd,MnPrv,Shed,0,-0.867605,1.645621,WD,Normal,142125,2156


In [12]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

def advanced_preprocess_categorical_columns(df, encoding='onehot', ordinal_columns=None, rare_threshold=10, unknown_categories='Other'):
    """Takes a Pandas DataFrame and performs advanced preprocessing on all categorical columns.
    
    Fills missing values with the mode, removes rare categories, and encodes the data using one-hot encoding or ordinal encoding.
    
    Args:
        df: Pandas DataFrame
        encoding: encoding method to use ('onehot' or 'ordinal')
        ordinal_columns: dictionary of ordinal columns and their corresponding categories (e.g., {'column_name': ['low', 'medium', 'high']})
        rare_threshold: threshold for removing rare categories (default is 10)
        unknown_categories: label for unknown categories (default is 'Other')
        
    Returns:
        modified DataFrame
    """
    # Make a copy of the DataFrame to avoid modifying the original data
    df = df.copy()
    
    # Get a list of all categorical columns
    categorical_columns = df.select_dtypes(include=['object']).columns
    
    # Fill missing values with the mode
    for col in categorical_columns:
        df[col] = df[col].fillna(df[col].mode()[0])
        
    # Remove rare categories
    for col in categorical_columns:
        # Get the counts of each category
        counts = df[col].value_counts()
        
        # Remove categories with less than rare_threshold occurrences
        df[col] = df[col].apply(lambda x: x if counts[x] >= rare_threshold else unknown_categories)
        
    if encoding == 'onehot':
        # Encode the data using one-hot encoding
        df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)
        
    elif encoding == 'ordinal':
        # Check if ordinal_columns is provided
        if ordinal_columns is None:
            raise ValueError("ordinal_columns must be provided when using ordinal encoding.")
            
        # Encode the data using ordinal encoding
        for col, categories in ordinal_columns.items():
            df[col] = df[col].astype('category', categories=categories, ordered=True)
            df[col] = df[col].cat.codes
            
    else:
        raise ValueError("Invalid encoding method. Must be 'onehot' or 'ordinal'.")
        
    print("Advanced preprocessing steps completed: filled missing values, removed rare categories, and encoded data.")
    return df

# Create a new dataframe to store the preprocessed data
preprocessed_df = pd.DataFrame()

# Preprocess the data and store it in the new dataframe
preprocessed_df = advanced_preprocess_categorical_columns(modified_df)


Advanced preprocessing steps completed: filled missing values, removed rare categories, and encoded data.


In [13]:
preprocessed_df.describe(include='all')

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,Fence_MnWw,MiscFeature_Shed,SaleType_New,SaleType_Other,SaleType_WD,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Other,SaleCondition_Partial
count,1430.000000,1430.000000,1430.000000,1430.000000,1430.000000,1430.000000,1430.000000,1430.000000,1430.000000,1430.000000,...,1430.000000,1430.000000,1430.000000,1430.000000,1430.000000,1430.000000,1430.000000,1430.000000,1430.000000,1430.000000
mean,731.976923,-0.066035,69.925874,10399.462937,6.123077,5.570629,1972.170629,1985.143357,104.951748,443.422378,...,0.007692,0.997203,0.084615,0.017483,0.867832,0.008392,0.013986,0.818881,0.002797,0.086713
std,421.546392,0.899626,22.156043,9174.696318,1.382268,1.105482,29.650783,20.554279,182.064376,456.694406,...,0.087398,0.052833,0.278406,0.131106,0.338792,0.091252,0.117474,0.385251,0.052833,0.281513
min,1.000000,-0.872563,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,368.250000,-0.872563,60.000000,7560.000000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,734.500000,-0.163109,69.000000,9458.500000,6.000000,5.000000,1974.000000,1994.000000,0.000000,383.500000,...,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,1096.750000,0.309859,79.000000,11604.500000,7.000000,6.000000,2001.000000,2004.000000,168.000000,707.500000,...,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,1460.000000,2.911188,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


To be Continued ....